In [1]:
%matplotlib notebook
import numpy as np
import cupy as qp
import operator as op
import matplotlib as mpl
from matplotlib import animation, rc
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive, Button
from IPython.display import display, HTML
import netCDF4 as nc

xp = np

In [2]:
# display functions

def disp3d(fig, aa, lines=(35,35)): # 3d wirefram plot
    # interpert inputs
    aa = qp.asnumpy(aa)
    
#     xlim = box[0]
#     ylim = box[1]
#     zlim = box[2]
#     if (xlim==None):
#         xlim = (0, aa[0].shape[0])
#     if (ylim==None): ylim = (0, aa[0].shape[1])
#     if (zlim==None):
#         ran = np.max(aa[0])-np.min(aa[0])
#         zlim = (np.min(aa[0])-ran, np.max(aa[0])+ran)
#         zlim = (-2, 2)
    
    #'wires' of the wireframe plot
    x = np.linspace(0, aa[0].shape[0]-1, lines[0], dtype=int)
    y = np.linspace(0, aa[0].shape[1]-1, lines[1], dtype=int)
    xx, yy = np.meshgrid(x, y, indexing='ij')
    
    #display it
#     fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
#     for a in aa:
    A = aa#[xx,yy]
    ax.plot_wireframe(xx, yy, A)
    
    return ax



# def genframes(a, frames=120):#np.arange(120, dtype=int)):
#     arts = np.array([])#np.empty((frames.shape[0],))
# #     assert frames.dtype==int, "frames must be a numpy array of integers"
# #     frames = np.asarray(frames, dtype=int)
#     mm = np.max([-np.min(a), np.max(a)])/2
#     ds = a[np.linspace(0, a.shape[0]-1, frames, dtype=int)]
#     for d in ds:
# #         d = np.asarray(a[frame], dtype=np.float32) # data
#         im = plt.imshow(d, animated=True, vmin=-mm, vmax=mm, cmap='seismic')
#         arts = np.append(arts, im)
#     return f

# def motioncon(fig, f): # animated height plot, takes in list of 2d height arrays
    #prepare figure/display
    
#     z = qp.asnumpy(f[0])
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
#     mm = np.max([-np.min(f), np.max(f)])/2
#     im = ax.imshow(z, vmin=-mm, vmax=mm,cmap='seismic')
#     cb = fig.colorbar(im)
#     tx = ax.set_title(title)
#     plt.xticks(np.linspace(0, z.shape[0], xlabels.shape[0]), xlabels)
#     plt.yticks(np.linspace(0, z.shape[1], ylabels.shape[0]), ylabels)
    
#     def animate(i): # returns i'th element (height array) in f
#         im.set_data(qp.asnumpy(f[i]))
#         plt.contour(h, levels=1, cmap='gray')
    
    #display it
#     anim = animation.ArtistAnimation(fig, f)
#     return anim

def vect(u, v, arws=(10, 10), arwsz=100): # vector /motion plot
    #interpert inputs
    u = qp.asnumpy(u)
    v = qp.asnumpy(v)
#     if (xlim=='default'): xlim = (0, u.shape[0])
#     if (ylim=='default'): ylim = (0, v.shape[1])
    arws = (int(arws[0]), int(arws[1]))
    
    # set up
    x = np.linspace(0, u.shape[0]-1, arws[0], dtype=int)
    y = np.linspace(0, v.shape[1]-1, arws[1], dtype=int)
    xx, yy = np.meshgrid(x, y, indexing='ij')
    uu = u[x,y]
    vv = v[x,y]
    m = np.hypot(uu, vv)
    
    #displat it
    q = plt.quiver(xx, yy, uu, vv, m, scale = 1/arwsz)
#     return ax
#     ax.set_xlim(xlim)
#     ax.set_ylim(ylim)
#     plt.title(title)
#     plt.show()

## Differentials of 2D arrays
Values live on a 2D grid.  
> Inputs:
> > a:  2D Matrix of values [N,M].  The matrix is ordered [i,j] so that increasing i, is increasing x, and increasing j is increasing y.<br>
        dx or dy: the width of the grid unit cell in some units.
    
> Evaluates:
> > derivative by subtracting adjacent values and dividing by the grid width.
As written dx and dy can either be constant scalar or an array that is (N-1,M) or (N,M-1) in size depending on if the derivative is on dx or dy, respectively. This allow for non-constant grid cell spacing as appropriate for equi-lattitude and longitude spaced grids.

> Outputs: 
> > the Returned array size is shorter by one along the direction (x or y) of the grid.  Effectively then the values represent the rate of change at points inbetween the neighboring points.  For the C-grid represnentaion the u,v speeds are values in between the n,h heights.


In [3]:
# useful math functions
def d_dx(a, dx):
    ddx = ( a[:-1] - a[1:] )*(-1/dx) 
    return ddx
def d_dy(a, dy):
    ddy = ( a[:,:-1] - a[:,1:] )*(-1/dy)
    return ddy
def div(u, v, dx, dy):
    div = d_dx(u, dx) + d_dy(v, dy)
    return div

# for generating simple environments or initial conditions
# def hydrodynamic
def planegauss(shape, win=((-2, 2), (-2, 2))):
   # h=np.empty(shape, dtype=np.float32)
    npx = np.linspace( win[0][0], win[0][1], shape[0] )
    npy = np.linspace( win[1][0],win[1][1], shape[1] )
    npxx, npyy = np.meshgrid(npx, npy, indexing='ij')
    h = np.exp( -np.e * ( npxx*npxx + npyy*npyy ) )
    return (h)
def lingauss(shape, w = 1/2, ax = 0, win = (-2, 2)):
   # h=np.empty(shape, dtype=np.float32)
    npx = np.linspace( win[0], win[1], shape[0] )
    npy = np.linspace( win[0], win[1], shape[1] )
    npxx, npyy = np.meshgrid(npy, npx)
    xy = (npyy, npxx)[ax]
    h = np.exp( -np.e * ( xy*xy ) / (w*w) )
    return (h)

## Class of objects to hold current state of an ocean grid 

Equations of motion
$$
\begin{align}
\frac{\partial n}{\partial t} & =
    -\frac{\partial  }{\partial x} \bigl( \left( n+h\right)u \bigr) 
    - \frac{\partial  }{\partial y}  \bigl( \left( n+h\right)v \bigr)\\  
\\
\frac{\partial u}{\partial t} & = Coriolis + Advection + Gravity\\
 & = +fv +\bigl( \kappa\nabla^{2}u - (u,v)\cdot\vec\nabla u \bigr)  
    - g\frac{\partial n}{\partial x}\\  
& = +fv +\bigl( \kappa\frac{\partial^{2} u}{\partial x^{2}}
           +\kappa\frac{\partial^{2} u}{\partial y^{2}}
           -u\frac{\partial u}{\partial x} - v\frac{\partial u}{\partial y}\bigr) 
           - g\frac{\partial n}{\partial x}\\
\\
\frac{\partial v}{\partial t} & = -fu 
   + \bigl( \kappa\nabla^{2}v - (u,v)\cdot\vec\nabla v \bigr) 
    - g\frac{\partial n}{\partial y}\\   
& = -fu+\bigl( \kappa\frac{\partial^{2} v}{\partial x^{2}}
           +\kappa\frac{\partial^{2} v}{\partial y^{2}}
           -u\frac{\partial v}{\partial x} - v\frac{\partial v}{\partial y}\bigr) 
           - g\frac{\partial n}{\partial y}\\           
\end{align}
$$

Where 
- *_h_* calm ocean depth (positive number) at any point. Presumed constant in time
- *_n_* is the wave height above the calm ocean height
- *_u_* is the mean water column velocity in the _x_ (east) direction
- *v* is the mean water column velocity in the _y_ (north) direction

and the physcial constant parameters are:
- *g* gravitational constant
- *f* is the lattidude dependent coriolis coefficient: $2\omega \sin(latitude)$
- $\kappa$ is the viscous damping coefficient across the grid cell boundaries


### Coriolis Force
$$
\begin{align}
U & = R \cos(\phi) O \\
u_i & = U+u_r \\
u_r & = u_i-U \\
\frac{\partial U}{\partial phi} & = -R \sin(\phi) O \\
        phi & = y/R \\
       \frac{\partial \phi}{\partial t}  & = v/R \\ 
        \frac{\partial U}{\partial t} & = v (-\sin(phi) O) \\
        \frac{\partial u_r}{\partial phi}  = \frac{\partial u_i}{\partial t} - \frac{\partial U}{\partial t} & = v O \sin(phi) \\
       \frac{\partial u_r}{\partial t} & = v O \sin(phi)      
\end{align}
$$

        

In [3]:
# physics constants
class p():
    g = np.float32(9.81) # gravity

class State(): # state
    def __init__(self, h, n, u, v, dx, dy, lat, lon):
        
        self.dx = dx
        self.dy = dy
        self.lat = lat
        self.lon = lon
#         self.lats, self.lons = np.meshgrid(self.lat, self.lon)
#         self.lat, self.lon = np.meshgrid(self.lat, self.lon) # lattitude/longitude chunk simulation area stretches over
        self.h = h
    
        self.maxws = np.sqrt(np.max(self.h)*p.g) # maximum wave speed
        
        self.n = np.asarray(n, dtype=np.float32) # surface height (eta)
        self.u = np.asarray(u, dtype=np.float32) # x vel array
        self.v = np.asarray(v, dtype=np.float32) # y vel array
        
        #make sure h is the same shap as n (eta)
        assert (np.isscalar(h) or self.h.shape == self.n.shape), "H and N must have the same shape, or H must be a scalar" # 'or' is short circuit
        
#         self.calcDt()
        self.dt = np.min((np.min(self.dx), np.min(self.dy)))/(4*self.maxws)
        
        self.coriolis = ((2*2*np.pi*np.sin(self.lat*np.pi/180))/(24*3600))[:, np.newaxis] # rotation speed of the earth dtheta/dt
        """ derivation of coriolis force
        U = R*cos(phi)*O
        ui = U+ur
        ur = ui-U
        dU/dphi = -R*sin(phi)*O
        phi = y/R
        dphi/dt = v/R
        dU/dt = v*(-sin(phi)*O)
        dur/dt = dui/dt - dU/dt = v*O*sin(phi)
        dur/dt = v*O*sin(phi)"""
        self.movetodevice()
    def movetodevice(self):
        self.lat = xp.asarray(self.lat)
        self.lon = xp.asarray(self.lon)
        self.h = xp.asarray(self.h)
        self.n = xp.asarray(self.n)
        self.u = xp.asarray(self.u)
        self.v = xp.asarray(self.v)
        self.coriolis = xp.asarray(self.coriolis)
#     def calcDt(self, fudge = 5): #calculate optimal value of dt for the height and dx values
#         dx = np.min(self.dx)
#         dy = np.min(self.dy)
#         self.dt = np.min((dx, dy))/(fudge*self.maxws)
props = op.itemgetter('h', 'n', 'u', 'v', 'dx', 'dy', 'lat', 'lon') # for grrabbing the elements of a state
def newstate(state):
    return State(*(props(vars(state))))


NameError: name 'np' is not defined

# physics shallow water framework
##  Equations of motion 
$$
\begin{align}
\frac{\partial n}{\partial t} & = -\frac{\partial  }{\partial x} \bigl( \left( n+h\right)u \bigr) - \frac{\partial  }{\partial y}  \bigl( \left( n+h\right)v \bigr)\\
\frac{\partial u}{\partial t} & = -u\frac{\partial u}{\partial x} - v\frac{\partial u}{\partial y} +fv - g\frac{\partial n}{\partial x}\\
\frac{\partial v}{\partial t} & = -u\frac{\partial v}{\partial x} - v\frac{\partial v}{\partial y} -fv - g\frac{\partial n}{\partial x}
\end{align}
$$

Where 
- *_h_* calm ocean depth (positive number) at any point. Presumed constant
- *_n_* is the wave height above the calm ocean height
- *_u_* is the mean water column velocity in the _x_ (east) direction
- *v* is the mean water column velocity in the _y_ (north) direction

and the physcial constant parameters are:
- *g* gravitational constant
- *f* is the lattidude dependent coriolis coefficient: $2\omega \sin(latitude)$


In [5]:
def dndt(h, n, u, v, dx, dy) : # for individual vars
# def dndt(state):
    """change in n per timestep, by diff. equations"""
#     h, n, u, v, dx, dy = [qp.asnumpy(state.__dict__[k]) for k in ('h', 'n', 'u', 'v', 'dx', 'dy')]
    hx = xp.empty((n.shape[0]+1, n.shape[1]), dtype=n.dtype) # to be x (u) momentum array
    hy = xp.empty((n.shape[0], n.shape[1]+1), dtype=n.dtype)
    hx[1:-1] = ((h+n)[1:] + (h+n)[:-1])/2 # average to same shape as u
    hx[0] = hx[-1] = 0.0 # reflective boundaries/borders
    hy[:,1:-1] = ((h+n)[:,1:] + (h+n)[:,:-1])/2
    hy[:,0] = hy[:,-1] = 0.0
#     print(hx.shape, u.shape)
    hx *= u # height/mass->momentum of water column.
    hy *= v
    dndt = (div(hx, hy, -dx, -dy))
    return ( dndt )
 # change in x vel. (u) per timestep
def dudt(n, f, u, v, dx, dy) : # for individual vars
# def dudt(state):
#     f, n, u, v, dx, dy = [qp.asnumpy(state.__dict__[k]) for k in ('coriolis', 'n', 'u', 'v', 'dx', 'dy')]
    #### Memory optimization possible here.  One could pass in the empty array pre-allocated from the calling
    #### subroutine.  As long as the size is not changing one can keep recylcling these dummy arrays in the outer loop
    #### thus avoiding allocating new arrays.  What happens to the old arrays that are dereferenced?  Naively, they
    #### just accumulate on the gargbage heap, filling up memory till python's garbage collection comes along and reclaims the memory
    #### it.  If memory allocations are not contiguous then the memory needs defragmentation to make it contiguous.
    #### However, that's slightly naive.  More sophisticated Garbage collection systems, work with the allocation system
    #### to optimistically re-use a chunk of memory that is Just-the-right-size that came from a recently dereferenced opbject.
    #### THat's a good bet much of the time when one is constantly allocating and deallocating things that are the same size.
    #### However, it's a balance between how clever the GC is and how clever you can be.  Sometimes it's better to do
    #### your own memory management using pre-allocated array that you recylce.
    #### The downside of that is your algorithms can't dynamically change the size of things in response to thier inputs.
    #### One way to ease that is to wrap the algorithm that uses the passed in caches with another function that dynamically
    #### creates them: that is move the "np.empty()" command into the wrapper, and pass it in.  Then you have the flexibility
    #### to call either the outer or inner function from your driver function.  You could even timeit.
    
    dudt = xp.empty((n.shape[0]+1, n.shape[1]), dtype=n.dtype) # x accel array
    #### using explicit indicies on the left side of assignment means the operation can be optimized without
    #### making a copy.  internally python will construct a copy of the rHS but can do it slightly more efficiently if
    #### it doesn't have to keep it after the = assignment is done.
    ####
    #### Minor optimization:  multiply the p.g unto the dx IF dx is a scalar.  If dx is a matrix it's not helpful.
    dudt[1:-1] = -p.g*d_dx(n, dx)
    
    dudt[1:-1] += ((f*v)[1:, 1:]+(f*v)[1:, :-1]+(f*v)[:-1, 1:]+(f*v)[:-1,:-1])*0.25 # coriolis force
    dudt[1:-1] -= u[1:-1]*(d_dx(u, dx)[1:] + d_dx(u, dx)[:-1])*0.5 # advection
    #### why not do this:
    ####      dux = d_dx(u, dx)
    ####      dudt[:,1:-1] = ( dux[:, 1:] + dux[:, :-1] ) * 0.5
    #### comment:  note that (d_dx(u, dx)[1:] + d_dx(u, dx)[:-1]) has middle terms that can drop out!
    #### thus it is equal to  U(i-1)-U(i)+U(i)-U(i+1) whenever dx is a constant.  if Dx is not a constant it's
    #### not true however.
    duy = xp.empty((n.shape[0]+1, n.shape[1]), dtype=n.dtype)
    dudy = d_dy(u, dy)
    duy[:,1:-1] = ( dudy[:, 1:] + dudy[:, :-1] ) * 0.5
    #### need to comment what the next line is doing or approximating.  I think it's saying sheers don't reduce velocity
    duy[:,0] = dudy[:,0]
    duy[:,-1] = dudy[:, -1]
    #### correct sign????????????
    dudt[1:-1] += (v[1:, 1:]+v[1:, :-1]+v[:-1, 1:]+v[:-1, :-1])*0.25*duy[1:-1] # advection
    
    dudt[0] = dudt[-1] = 0 # reflective boundaries
    return ( dudt )

def dvdt(n, f, u, v, dx, dy) :
# def dvdt(state):
#     f, n, u, v, dx, dy = [qp.asnumpy(state.__dict__[k]) for k in ('coriolis', 'n', 'u', 'v', 'dx', 'dy')]
    #
    dvdt = xp.empty((n.shape[0], n.shape[1]+1), dtype=n.dtype)
    dvdt[:,1:-1] = -p.g*d_dy(n, dy)
    #### Minor optimization (-f*0.25)*  will be faster since it moves the minus and * into scalars. 
    #### comment that this line is different because f is negative for v
    dvdt += - f*(u[1:, 1:]+u[1:, :-1]+u[:-1, 1:]+u[:-1,:-1])*0.25 # coriolis
    
    #### why not this instead:
    ####      dvy = d_dy(u, dy)
    ####      dvdt[:,1:-1] = ( dvy[:, 1:] + dvy[:, :-1] ) * 0.5
    dvdt[:,1:-1] -= v[:,1:-1]*(d_dy(v, dy)[:,1:] + d_dy(v, dy)[:,:-1])*0.5 # advection
    dvx = xp.empty((n.shape[0], n.shape[1]+1), dtype=n.dtype)
    dvdx = d_dx(v, dx)
    dvx[1:-1] = ( dvdx[1:] + dvdx[:-1] ) * 0.5

    dvx[0] = dvdx[0]
    dvx[-1] = dvdx[-1]
    dvdt[:,1:-1] -= (u[1:, 1:]+u[1:, :-1]+u[:-1, 1:]+u[:-1, :-1])*0.25*dvx[:,1:-1] # advection
    
    dvdt[:,0] = dvdt[:,-1] = 0
    return ( dvdt )

def land(h, u, v, thresh = 50): # how to handle land/above water area
    #boundaries / land
    coastx = xp.less(h, thresh) # start a little farther than the coast so H+n is never less than zero
    ####  This is clever.  But anytime something is clever you should comment it carefully, as it not only helps
    #### the reader but also prevents you from out-clevering yourself.
    
    (u[1:])[coastx] = (u[:-1])[coastx] = 0 # set vel. on either side of land to zero, makes reflective
    (v[:,1:])[coastx] = (v[:,:-1])[coastx] = 0
    
    #####  QUESTION::::  COMMENT ON THIS
    #####  WHAT KEEPS n[coastx] from increasing?  Should it? maybe.  
    #####  Should one also set n(coasts) to something?  what?
    return (u, v)

In [6]:
def forward(h, n, u, v, f, dt, dx, dy, doland, beta=0): # forward euler and forward/backward timestep
    # beta = 0 - forward, beta = 1 - forward-backward
    n1 = n + ( dndt(h, n, u, v, dx, dy) )*dt
    u1 = u + ( beta*dudt(n1, f, u, v, dx, dy) +  (1-beta)*dudt(n, f, u, v, dx, dy) )*dt
    v1 = v + ( beta*dvdt(n1, f, u, v, dx, dy) +  (1-beta)*dvdt(n, f, u, v, dx, dy) )*dt
    u1, v1 = doland(h, u1, v1) # handle any land in the simulation
    return n1, u1, v1

def fbfeedback(h, n, u, v, f, dt, dx, dy, doland, beta=1/3, eps=2/3): # forward backward feedback timestep
    n1g, u1g, v1g = forward(h, n, u, v, f, dt, dx, dy, doland, beta) # forward-backward first guess
    #feedback on guess
    n1 = n + 0.5*(dndt(h, n1g, u1g, v1g, dx, dy) + dndt(h, n, u, v, dx, dy))*dt
    u1 = u + 0.5*(eps*dudt(n1, f, u, v, dx, dy)+(1-eps)*dudt(n1g, f, u1g, v1g, dx, dy)+dudt(n, f, u, v, dx, dy))*dt
    v1 = v + 0.5*(eps*dvdt(n1, f, u, v, dx, dy)+(1-eps)*dvdt(n1g, f, u1g, v1g, dx, dy)+dvdt(n, f, u, v, dx, dy))*dt
    u1, v1 = doland(h, u1, v1) # how to handle land/coast
    return n1, u1, v1


# versions using STATE class

# def forward(state, doland, beta=0): # forward euler and forward/backward timestep
#     # beta = 0 - forward, beta = 1 - forward-backward
#     h, n, u, v, dx, dy, dt = [qp.asnumpy(state.__dict__[k]) for k in ('h', 'n', 'u', 'v', 'dx', 'dy', 'dt')]
#     nstate = newstate(state)
#     nstate.n = n + ( dndt(state) )*dt
#     u1 = u + ( beta*dudt(nstate) +  (1-beta)*dudt(state) )*dt
#     v1 = v + ( beta*dvdt(nstate) +  (1-beta)*dvdt(state) )*dt
#     u1, v1 = doland(state.h, u1, v1) # handle any land in the simulation
#     return nstate.n, u1, v1


# def fbfeedback(state, doland, beta=1/3, eps=2/3): # forward backward feedback timestep
#     h, n, u, v, dx, dy, dt = [qp.asnumpy(state.__dict__[k]) for k in ('h', 'n', 'u', 'v', 'dx', 'dy', 'dt')]
#     pstate = newstate(state) # predictor state, rough approx. with forward/backward
#     pstate.n, pstate.u, pstate.v = forward(state, doland, beta) # forward-backward first guess
#     #feedback on guess
#     nstate = newstate(state)
#     nstate.n = n + 0.5*(dndt(pstate) + dndt(state))*dt
#     u1 = u + 0.5*(eps*dudt(nstate)+(1-eps)*dudt(pstate)+dudt(state))*dt
#     v1 = v + 0.5*(eps*dvdt(nstate)+(1-eps)*dvdt(pstate)+dvdt(state))*dt
#     u1, v1 = doland(state.h, u1, v1) # how to handle land/coast
#     return nstate.n, u1, v1



def timestep(h, n, u, v, f, dt, dx, dy): return fbfeedback(h, n, u, v, f, dt, dx, dy, land) # switch which integrator/timestep is in use
# def timestep(state): return fbfeedback(state, land) # which integrator/timestep is in use

#

In [7]:
def simulate(initstate, t, ffs = 120): # gives surface height array of the system after evert dt
    h, n, u, v, dx, dy, dt = [qp.asnumpy(initstate.__dict__[k]) for k in ('h', 'n', 'u', 'v', 'dx', 'dy', 'dt')]#h, state.n, state.u, state.v, state.dx, state.dy, state.dt
    state = initstate
    assert (dt >= 0), 'negative dt!' # dont try if timstep is zero or negative
#     ff = itrs//1000 + 1 # frequency of snapshots taken into time array, to prevent memory error
    nt = np.empty((ffs,)+n.shape, dtype=n.dtype) # list of height arrays over time # (int(np.ceil(itrs/100)) #int(np.ceil(itrs/ff))
    maxn = np.zeros(n.shape, dtype=n.dtype) # max height in that area
    minn = np.zeros(n.shape, dtype=n.dtype) # minimum height that was at each point
    timemax = np.zeros(n.shape, dtype=n.dtype) # when the maximum height occured
    ind = 0
    itrs = int(np.ceil(t/dt))
    ff = itrs/(ffs) # number of iterations between frames
#     for af in range(ffs):
    for itr in range(itrs):# iterate for the given number of iterations
        n, u, v = timestep(h, n, u, v, state.coriolis, dt, dx, dy) # pushes n, u, v one step into the future
        n = qp.asnumpy(state.n)
        maxn = np.max((n, maxn), axis=0) # record new maxes if they are greater than previous records
        minn = np.min((n, minn), axis=0)
        timemax[np.greater(n, maxn)] = itr*dt
        nt[int(np.floor(itr/ff))] = qp.asnumpy(n) # add a new snapshot of the surface height for every frame # every ff iteration
        print('['+int(np.round(30*itr/itrs))*'='+'>'+ int(30-np.round(30*itr/itrs))*' '+'] ' \
               + str(np.round(100*itr/itrs, 3))+'% | '+str(np.round(itr*dt, 3))+' of '+str(t)+' seconds ', end='\r')
        #             ind+=1# add new surface height to array
    print()
    print('simulation finished')
    return {'frames':nt, 'max':maxn, 'min':minn, 'time':timemax} # return surface height through time and maximum heights

# verifaction

In [8]:
#wavespeed and differential tests
import unittest
fooo = []
class testWaveSpeed(unittest.TestCase): # tests if the wave speed is correct
    def setUp(self):
        self.dur = 500 # duration of period to calculate speed over
        self.size = (10, 1000) # grid squares (dx's)
        self.dx = np.single(100, dtype=np.float32) # meters
        self.dy = np.single(100, dtype=np.float32)
        self.lat = np.linspace(0, 0, self.size[0]) # physical location the simulation is over
        self.lon = np.linspace(0, 0 , self.size[1])
        self.h = 100
        self.n = 1*lingauss(self.size, 1/4, 1) # intial condition single wave in the center
        self.u = np.zeros((self.size[0]+1, self.size[1]+0)) # x vel array
        self.v = np.zeros((self.size[0]+0, self.size[1]+1)) # y vel array
        self.margin = 0.1 # error margin of test
    def calcWaveSpeed(self, ar1, ar2, Dt): # calculat how fast the wave is propagating out
        midstrip1 = ar1[int(ar1.shape[0]/2),int(ar1.shape[1]/2):]
        midstrip2 = ar2[int(ar1.shape[0]/2),int(ar2.shape[1]/2):]
        peakloc1 = np.argmax(midstrip1)
        peakloc2 = np.argmax(midstrip2)
        plt.figure(1)
        plt.clf()
        plt.plot(midstrip1)
        plt.plot(midstrip2)
        plt.show()
        speed = (peakloc2 - peakloc1)*self.dy/Dt
        return speed
    def calcExactWaveSpeed(self): # approximently how fast the wave should be propagating outwards
        ws = np.sqrt(p.g*np.average(self.h))
        return ws
    def test_wavespeed(self): # test if the expected and calculated wave speeds line up approcimently
        self.testStart = State(self.h, self.n, self.u, self.v, self.dx, self.dy, self.lat, self.lon)
        self.simdata = simulate(self.testStart, self.dur)
#         self.testFrames, self.testmax, self.testmin = self.simdata[:3]
        self.testFrames = self.simdata['frames']
        self.testEndN = self.testFrames[-1]
        calcedws = self.calcWaveSpeed( self.testStart.n, self.testEndN, self.dur )
        exactws = self.calcExactWaveSpeed()
        err = (calcedws - exactws)/exactws
        print(err, self.margin)
        assert(abs(err) < self.margin) # error margin
    def tearDown(self):
        del(self.dur)
        del(self.dx)
        del(self.dy)
        del(self.lat)
        del(self.lon)
        del(self.size)
        del(self.h)
        del(self.n)
        del(self.u)
        del(self.v)

class testdifferential(unittest.TestCase): # differental function test (d_dx)
    def setUp(self):
        self.a = np.arange(144) # test input
        self.a = self.a.reshape(12, 12) # make into 2d array
        self.ddthreshold = 1E-16
    def test_ddx(self):
        da = d_dx(self.a, 1)
        diff = np.abs(da[1:-1] - np.mean(da[1:-1]))
        maxdiff = np.max(diff)
        self.assertTrue(np.all(np.abs(da[-1:1] < self.ddthreshold)),"expected zero along borders")
        self.assertTrue(np.all(diff < self.ddthreshold),"Expected constant d_dx less than %f but got %f"%(self.ddthreshold,maxdiff))
    def tearDown(self):
        del(self.a)
        del(self.ddthreshold)

unittest.main(argv=['first-arg-is-ignored'], exit=False)
#You can pass further arguments in the argv list, e.g.
#unittest.main(argv=['ignored', '-v'], exit=False)      
#unittest.main()

E.
ERROR: test_wavespeed (__main__.testWaveSpeed)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-8-88961b44bfb7>", line 34, in test_wavespeed
    self.simdata = simulate(self.testStart, self.dur)
  File "<ipython-input-7-016be3195103>", line 15, in simulate
    n, u, v = timestep(h, n, u, v, state.coriolis, dt, dx, dy) # pushes n, u, v one step into the future
  File "<ipython-input-6-d981b95f8ced>", line 46, in timestep
    def timestep(h, n, u, v, f, dt, dx, dy): return fbfeedback(h, n, u, v, f, dt, dx, dy, land) # switch which integrator/timestep is in use
  File "<ipython-input-6-d981b95f8ced>", line 10, in fbfeedback
    n1g, u1g, v1g = forward(h, n, u, v, f, dt, dx, dy, doland, beta) # forward-backward first guess
  File "<ipython-input-6-d981b95f8ced>", line 5, in forward
    v1 = v + ( beta*dvdt(n1, f, u, v, dx, dy) +  (1-beta)*dvdt(n, f, u, v, dx, dy) )*dt
  File "<ipython-input-5-39ed778c4077>",

In [9]:
simpletestcase = {
    'h': 100,#*np.ones((100, 100), dtype=np.float),
    'n': planegauss((100, 100)),
    'u': np.zeros((101, 100)),
    'v': np.zeros((100, 101)),
    'dx': 100,
    'dy': 100,
    'lat': np.zeros((100,)),
    'lon': np.zeros((100,))
}
simpleState = State(**simpletestcase)
simpleframes = simulate(simpleState, 100, 100)['frames']#, simpleMax, simpleMin, simpleAT

# fig = plt.figure(23)
# plt.imshow(simpleframes[50])

fig = plt.figure(26)

simpleart = [(plt.imshow(simplef),) for simplef in simpleframes]
anim = animation.ArtistAnimation(fig, simpleart, interval=50, blit=True, repeat_delay=1000)
plt.show()

ValueError: operands could not be broadcast together with shapes (100,101) (100,99) (100,101) 

## indonesian tsunami

In [ ]:

##### It would be perhaps better to have an event class, then read in a YAML string to populate it for
##### different events like Palu, Caribean and Sumatra
##### You can then have a smaller driver class for each case which will hold the event class but also provide
##### any custom evaluations special to those cases that are different about them.
##### That is abstract the similar features into an event class.  THen customize that for 

class indone2004():
    event = {
        'lat': 3.316,
        'lon': 95.854
    } # source of the tsunami
    
    dlat = 111000 # latitude degree to meters
    psize = (dlat*30*np.cos(22.5*np.pi/180), dlat*15) # physical size of area
    size = (2500, 1250) # grid squares (dx) # lat, lon

    dx = np.single(psize[0]/size[0], dtype=np.float32) # meters
    dy = np.single(psize[1]/size[1], dtype=np.float32) # meters

    
    bath = nc.Dataset('../data/bathymetry.nc','r')
    
    rxy = (8, 16)
    lat = bath.variables['lat'][:]#[latin]
    lon = bath.variables['lon'][:]
    latr = (np.abs(lat-event['lat']+rxy[1]).argmin(), np.abs(lat-event['lat']-rxy[1]).argmin())
    lonr = (np.abs(lon-event['lon']+rxy[0]).argmin(), np.abs(lon-event['lon']-rxy[0]).argmin())
    latin = np.linspace(latr[0], latr[1], size[0], dtype=int)
    lonin = np.linspace(lonr[0], lonr[1], size[1], dtype=int) # indexes of the bathymetry dataset we need
    lat = bath.variables['lat'][latin]
    lon = bath.variables['lon'][lonin]
    h = np.asarray(-bath.variables['elevation'][latin, lonin], dtype=np.float32)

    n = np.zeros(size)

    evinlat = np.argmin(np.abs(lat - event['lat']))
    evinlon = np.argmin(np.abs(lon - event['lon'])) # the index of the closest value to the correct longitude
    rady = 1+2*(int(25000/dy)//2) # number of indicies across the disturbance is
    radx = 1+2*(int(25000/dx)//2) # modified to be odd, so a point lands on the max of the gaussian
#     evpatch = \

#     evpatch = \
    n[evinlat-rady:evinlat+rady, evinlon-radx:evinlon+radx] = \
    50*planegauss((2*rady, 2*radx))

    u = np.zeros((size[0]+1, size[1]+0))
    v = np.zeros((size[0]+0, size[1]+1))

indonesia = State(*(props(vars(indone2004))))

In [ ]:
seasurface = qp.asnumpy(indonesia.n)#np.empty(qp.asnumpy(indonesia.n).shape)
# seasurface[::-1] = qp.asnumpy(indonesia.n)
bathymetry = -qp.asnumpy(indonesia.h)


plt.figure(116)
plt.title('initial conditions of indonesia simulation')

# plt.subplot(121)
a1 = plt.imshow(seasurface, cmap='seismic', vmin=-np.max(seasurface), vmax=np.max(seasurface))
tt1 = plt.title('initial sea surface height')
cb1 = plt.colorbar()
cb1.set_label('sea surface height (m)')

# plt.subplot(222)
a2 = plt.contour(-bathymetry, cmap='Greys')
tt2 = plt.title('bathymetry')
cb2 = plt.colorbar()
cb2.set_label('ocean depth (m)')

# plt.subplot(122)
# # a3 = vect()
# tt3 = plt.title('inital velocity (m/s)')


In [ ]:
indosim = simulate(indonesia, 2500)
indot = indosim['frames']
maxindo = indosim['max']
minindo = indosim['min']
tmindo = indosim['time']

masq = np.zeros(qp.asnumpy(indonesia.h).shape, dtype=qp.asnumpy(indonesia.h).dtype)
runuplocs = np.array([(5.251, 95.253), (5.452, 95.242), (5.389, 95.960), (2.575, 96.269), (4.208, 96.040)])
radx = masq.shape[0]//100
rady = masq.shape[1]//100
for runuploc in runuplocs:
    arglat, arglon = np.argmin(np.abs(indonesia.lat-runuploc[0])), np.argmin(np.abs(indonesia.lon-runuploc[1])) # gat the index location of this event
    masq[arglat-radx:arglat+radx, arglon-rady:arglon+rady] = 1 # make a small blip around the locatoin

plt.figure(123)
plt.contour(masq)
plt.show()

In [ ]:
# shallowWater/data/2004indonesiarunups.txt

In [ ]:
indof = np.transpose(indot, (0, 1, 2))
print(indof.shape)
maxt = np.max(indof,axis=(1,2))
print(maxt.shape)
imaxt = np.float32(1.0/maxt)
# plt.figure(888)
# plt.semilogy(maxt)
# plt.show()
norm_indof = indof*imaxt[:,np.newaxis,np.newaxis]

h = qp.asnumpy(indonesia.h)
# ht = np.transpose(h)
ht = h

In [ ]:
fig = plt.figure(122)

# plt.subplot(1, 3, 1)
plt.title('movie')
# f = genframes(norm_indof*0.3, frames=np.linspace(0, norm_indof.shape[0]-1, 300, dtype=int))
indoArts = [(plt.imshow(normindoframe, cmap='seismic', vmin=-np.max(normindoframe), vmax=np.max(normindoframe)),) for normindoframe in norm_indof]
indoAnim = animation.ArtistAnimation(fig, indoArts)
cb = plt.colorbar()
cbtt = cb.set_label('sea surface height (m)')


coast = plt.contour(ht, colors='black', levels=1)#, levels=3)
# locmask = plt.contour(masq, colors='green', levels=2)

In [ ]:
fig = plt.figure(124)
plt.title('maximum')
plt.imshow(maxindo+1, cmap='seismic', norm=mpl.colors.LogNorm())
# plt.colorbar()
plt.contour(ht-20, colors='black', levels=1)

In [ ]:
fig = plt.figure(126)
plt.title('minimum')
plt.imshow(1-minindo, cmap='seismic', norm=mpl.colors.LogNorm())
    # plt.colorbar()
plt.contour(ht-20, colors='black', levels=1)

plt.show()

# comparing indonesia sim to real data

In [ ]:
import pandas as pd

ff = pd.read_csv('~rrs/shallowWater/data/2004indonesiarunups2.txt',sep='\t')
#with open('~rrs/shallowWater/data/2004indonesiarunups.txt','r') as f:
#     txt = f.read()
ff   

In [ ]:
# includes the whole world
allruns = pd.read_csv('~rrs/shallowWater/data/al2004indorunups2.txt', sep='\t')
allruns = allruns.dropna(how='any',subset=['Latitude', 'Longitude', 'TTHr', 'TTMin', 'MaxWaterHeight'])
allruns = allruns.loc[:,['Latitude','Longitude','MaxWaterHeight','DistanceFromSource','TTHr','TTMin','Name']]

In [ ]:
# import data from file
indorut = pd.read_csv('~rrs/shallowWater/data/2004indonesiarunups2.txt',sep='\t')
indoevents = list(indorut.transpose().to_dict().values()) # list of dicts of events' properties
indoevents

In [ ]:
# sort the events by proximity to the source
# indoevents.sort(key = lambda event: np.sqrt((indone2004.event['lat'] - event['Latitude'])**2 + (indone2004.event['lat'] - event['Longitude'])**2))
indoevents.sort(key = lambda event: event['DistanceFromSource'])

In [ ]:
# simindomaxh = np.array([], dtype=np.float32) # the list of the maximum height that occured at each location
# for event in indoevents: # get the maximum water height at each location in the sim in the order given to us
# #     print(event)
#     evlat = event['Latitude'] # the latitude of the measurement
#     evlon = event['Longitude'] # "  longitude " "       "
#     argevlat = np.argmin(np.abs(indonesia.lat-evlat)) # the array index with latitude closest to point
#     argevlon = np.argmin(np.abs(indonesia.lon-evlon)) # the array index with longitude closest to point
#     mh = maxindo[argevlat, argevlon] # the maximum height at this point
#     if (argevlat == 0 or argevlon == 0): # if the point is out of the map so it just returns the edge
#         mh = 0 # ignore the value at the edge 
#     simindomaxh = np.append(simindomaxh, mh)

# list of indexes maximum height at the location of each event
# simindomaxh = [np.max(\
#                       maxindo[\
#                               np.argmin(np.abs(indonesia.lon-event['Longitude'])), \
#                               np.argmin(np.abs(indonesia.lat-event['Latitude']))]) \
#                for event in indoevents]
def simindomaxhgenerator():
    for event in indoevents:
        ilon = np.argmin(np.abs(indonesia.lon-event['Longitude']))
        ilat = np.argmin(np.abs(indonesia.lat-event['Latitude']))
        rad = 20
        ilon = np.max((ilon, rad))
        ilat = np.max((ilat, rad))
        
#         print(maxindo.shape, maxindo[ilon-rad:ilon+rad, ilat-rad:ilat+rad].shape, [ilon-rad,ilon+rad, ilat-rad,ilat+rad])
        try:
            mh = np.max(maxindo[ilon-rad:ilon+rad, ilat-rad:ilat+rad])
        except (ValueError): # when the section is entirely off the map so it returns an empty array
            mh = -1
        yield mh

simindomaxh = np.asarray(list(simindomaxhgenerator()))
simindomaxh

In [ ]:
indomaxheights = np.array([event['MaxWaterHeight'] for event in indoevents]) # the max water heights of each of the events in the order they are listed

In [ ]:
#doesn't work - times not recorded
# indotimemh = [event['TTHr']*3600+event['TTMin']*60 for event in indoevents] # a 2d map of the time it took to get there
# indotimemh

In [ ]:
plt.figure(133)
plt.plot([event['DistanceFromSource'] for event in indoevents])
plt.show()

In [ ]:
plt.figure(236)
datamap = np.greater(simindomaxh, 0)
print(datamap)
plt.title('maximum heights at various locations')

plt.plot(indomaxheights[datamap], simindomaxh[datamap])

# plt.plot(indomaxheights[datamap], label='measurement') # real data of max water heights

# plt.plot(simindomaxh[datamap], label='simulation') # simulation max water heights in the same order

plt.legend()
plt.show()

# Palu event

In [ ]:

class paluClass():

    dlat = 111000 # latitude degree to meters
    psize = (dlat*30*np.cos(22.5*np.pi/180), dlat*15) # physical size of area
    size = (400, 400) # grid squares (dx)

    dx = np.single(psize[0]/size[0], dtype=np.float32) # meters
    dy = np.single(psize[1]/size[1], dtype=np.float32) # meters

    
    bath = nc.Dataset('../data/bathymetry.nc','r')
    loc = (0, 120)
    rad = (1, 1)
    trx = (np.argmin(np.abs(bath.variables['lat'][:]-loc[0]-rad[0])), np.argmin(np.abs(bath.variables['lon'][:]-loc[1]-rad[1])))
    blx = (np.argmin(np.abs(bath.variables['lat'][:]-loc[0]+rad[0])), np.argmin(np.abs(bath.variables['lon'][:]-loc[1]+rad[1])))
    latin = np.linspace(trx[0], blx[0], size[0], dtype=int) # lon 118, 122, lat -2, 2
    lonin = np.linspace(trx[1], blx[1], size[1], dtype=int) # indexes of the bathymetry dataset we need
    lat = bath.variables['lat'][latin]
    lon = bath.variables['lon'][lonin]
    h = np.asarray(-bath.variables['elevation'][latin, lonin], dtype=np.float32)[:, ::-1]

    n = np.zeros(size)
    

    u = np.zeros((size[0]+1, size[1]+0))
    v = np.zeros((size[0]+0, size[1]+1))

def paluEvent(event):
    pev = paluClass
#     print(vars(paluClass)
    y = np.argmin(np.abs(pev.lat-event['lat']))
    x = np.argmin(np.abs(pev.lon-event['lon']))
    ry = 1+2*(int(250000/pev.dy)//2) # number of indicies across the disturbance is
    rx = 1+2*(int(250000/pev.dx)//2) # modified to be odd, so one point lands on tip og gaussian
    pev.n[y-ry:y+ry, x-rx:x+rx] = 50*planegauss((2*ry, 2*rx))
    pstate = State(*(props(vars(pev))))
    return pstate
palu = paluEvent({'lat':-0.2, 'lon':120.2}) # palu, indonesia recent tsunami
# palu = State(*(props(vars(ev))))

In [ ]:
# seasurface = np.empty(qp.asnumpy(palu.n).shape)
seasurface = qp.asnumpy(palu.n)
bathymetry = -qp.asnumpy(palu.h)


plt.figure(162)
plt.title('initial conditions of indonesia simulation')

# plt.subplot(121)
a1 = plt.imshow(seasurface, cmap='seismic', vmin=-np.max(seasurface), vmax = np.max(seasurface))
tt1 = plt.title('initial sea surface height with traced coastline')
cb1 = plt.colorbar()
cb1.set_label('sea surface height (m)')

# plt.subplot(222)
a2 = plt.contour(bathymetry, levels=1, colors='black')


# plt.subplot(122)
# # a3 = vect()
# tt3 = plt.title('inital velocity (m/s)')

In [ ]:
palusim = simulate(palu, 25000)

In [ ]:
# palut, maxpalu, minpalu, tmpalu = palusim[:4]
palut = palusim['frames']
maxpalu = palusim['max']
minpalu = palusim['min']
timepalu = palusim['time']

In [ ]:
paluf = np.transpose(palut, (0, 1, 2))
print(paluf.shape)
maxt = np.max(paluf,axis=(1,2))
print(maxt.shape)
imaxt = np.float32(1.0/maxt)
# plt.figure(888)
# plt.semilogy(maxt)

# plt.show()
norm_paluf = paluf*imaxt[:,np.newaxis,np.newaxis]

fig = plt.figure(167)
ttl = plt.title('animation')
f = [(plt.imshow(pframe, cmap='seismic', vmin=-np.max(pframe), vmax=np.max(pframe)),) for pframe in palut]
# f = genframes(norm_paluf*0.3)#, frames=np.linspace(0, norm_paluf.shape[0]-1, 300, dtype=int))
anim = animation.ArtistAnimation(fig, f)
cb = plt.colorbar()
h = qp.asnumpy(palu.h)
# ht = np.transpose(h)
ht = h
coast = plt.contour(ht-20, colors='black', levels=1)

fig = plt.figure(169)
plt.subplot(1, 2, 1)
plt.title('maximum')
plt.imshow(maxpalu+1, cmap='seismic', norm=mpl.colors.LogNorm())
# plt.colorbar()
plt.contour(ht-20, colors='black', levels=1)

plt.subplot(1, 2, 2)
plt.title('minimum')
plt.imshow(1-minpalu, cmap='seismic', norm=mpl.colors.LogNorm())
# plt.colorbar()
plt.contour(ht-20, colors='black', levels=1)

plt.show()